# Retrain the YOLO model

To retrain the YOLO model we need a prepared dataset of car images with moderate and severe accident labels.  We have such a dataset (obtained from RoboFlow) that already contains annotated images, splitted into training and validation datasets.  We will use these training/validation sets to retrain our current YOLO model. A few info on the data structure needed for training:

1. The encode classes of objects we want to teach our model to detect is 0-`moderate` and 1-`severe`.
2. The dataset will be in its own folder, with 2 subfolders in it: `train` and `valid`.  Within each subfolder there will be 2 subfolders: `images` and `labels`.
3. Each image has a corresponding annotation text file in the `labels` subfolder. The annotation text files have the same names as the image files.
4. A dataset descriptor YAML file (data.yaml) points to the datasets and describes the object classes in them. This YAML file is passed to the `train` method of the model to start the training process.

Let's get started!

In [ ]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

import os
import requests
import zipfile
from tqdm.notebook import tqdm
from ultralytics import YOLO

Next let's load a YOLO model 'yolo8m.pt'

In [ ]:
# Load model
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

## Get the training data

We have provided the following 2 training data sets, available as zip files:  
1) `accident-full.zip`   - to be used to fully re-train the model.
2) `accident-sample.zip` - to be used to partially re-train the model when we don't have the time to fully re-train the model.

During the workshop we will only use the sample dataset.

In [ ]:
# Function to retrieve a specific dataset
def retrieve_dataset(dataset_type):

    # Check if the directory exists, if not, create it
    if not os.path.exists("./datasets/"):
        os.makedirs("./datasets/")

    URL = f"https://rhods-public.s3.amazonaws.com/sample-data/accident-data/accident-{dataset_type}.zip"

    # Check if the file exists, if not, download and unzip it
    if not os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Downloading file...")
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(f'./datasets/accident-{dataset_type}.zip', 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    if os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Unzipping file...")
        with zipfile.ZipFile(f'./datasets/accident-{dataset_type}.zip', 'r') as zip_ref:
            zip_ref.extractall(path='./datasets/')
    print("Done!")


dataset_type = 'sample'
# dataset_type = 'full' # Use this line instead if you want to retrieve the full dataset
retrieve_dataset(dataset_type)

## Re-training our YOLO model

Let's start by understanding what an 'epoch' is.  Machine learning models are trained with specific datasets passed through an algorithm. Each time the full dataset passes through the algorithm, it is said to have completed an **epoch**. Each **epoch** will further refine the training of the model.

In the training code below we will set various parameters:  
**results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)**

- epochs: as we want to only demo the process, we will use only 1 epoch.
- imgsz: this is the size of the images the model has to be fed with.
- batch: this is the number of images that the algorithm will process simultaneously. The more the better results, but the more memory it consumes. As we are on constrained resources in this workshop, and this is only a training example, we keep it intentionally low at 2.

In your training run, each **epoch<** will show a summary for both the training and validation phases: lines 1 and 2 show results of the training phase and lines 3 and 4 show the results of the validation phase for each epoch.  

Execute the following cell to start re-training the model!

In [ ]:
# Train the model

results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)

**Note**: after a standard re-training, if we are happy with the results, we could export our model to the ONNX format. 
(you would replace trainX by the traininig session you want to use in the following command)

`ObjDetOXModel = YOLO("runs/detect/trainX/weights/best.pt").export(format="onnx")`

## Interpreting our Training Results

A full description of the process and the results for a training against the **full dataset** is available in the Lab instructions.

Now that we have retrained our model let's test it against images with car accidents!

**Please open the notebook `04-04-accident-recog.ipynb`**.